import pandas as pd

In [1]:
import pandas as pd

In [3]:
kyrenia = pd.read_csv(r"C:\Users\Dell\Desktop\Attractions\Kyrenia Gate.csv", error_bad_lines=False);
gul_hanim = pd.read_csv(r"C:\Users\Dell\Desktop\Hotel\Gul Hanim House Boutique Hotel.csv", error_bad_lines=False);
cypriot = pd.read_csv(r"C:\Users\Dell\Desktop\Hotel\Cypriot Swallow Boutique Hotel.csv", error_bad_lines=False);


tripadv_review = pd.concat([kyrenia, gul_hanim, cypriot])
tripadv_review

,r_points,r_text,place
0,50,Place is not bad but it's disappointment becau...,Kyrenia Gate
1,40,The North Nicosia Tourist Info Office is locat...,Kyrenia Gate
2,40,The Kyrenia Gate (it may be noted as the Girne...,Kyrenia Gate
3,40,Another old tourist attraction. No need to spe...,Kyrenia Gate
4,20,The northern main entrance to Nicosia with its...,Kyrenia Gate
...,...,...,...
26,50,I don't know how l can start to say something ...,Cypriot Swallow Boutique Hotel
27,50,We enjoyed a day sightseeing in Lefkosa (North...,Cypriot Swallow Boutique Hotel
28,40,"This is a nice, small hotel in the north of th...",Cypriot Swallow Boutique Hotel
29,50,One of the few cheaper hotels/apartments that ...,Cypriot Swallow Boutique Hotel


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0     [place, disappointment, small, different, fant...
1     [north, nicosia, tourist, info, office, locate...
2     [kyrenia, gate, noted, girne, gate, entrances,...
3     [tourist, attraction, need, spend, time, enjoy...
4     [northern, main, entrance, nicosia, ancient, g...
                            ...                        
26    [know, start, place, experience, place, funny,...
27    [enjoyed, sightseeing, lefkosa, north, cyprus,...
28    [nice, small, hotel, north, city, little, hard...
29    [cheaper, hotels, apartments, north, nicosia, ...
30    [stayed, night, swallow, boutique, hotel, nico...
Name: r_text, Length: 117, dtype: object

In [9]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=150,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [19]:
from pprint import pprint

In [20]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"hotel" + 0.015*"nicosia" + 0.012*"breakfast" + 0.010*"city" + '
  '0.010*"turkish" + 0.009*"clean" + 0.008*"rooms" + 0.007*"stay" + '
  '0.007*"great" + 0.007*"place"'),
 (1,
  '0.022*"gate" + 0.015*"nicosia" + 0.012*"kyrenia" + 0.011*"hotel" + '
  '0.011*"good" + 0.010*"room" + 0.010*"city" + 0.009*"stay" + 0.009*"place" + '
  '0.009*"breakfast"'),
 (2,
  '0.014*"house" + 0.012*"place" + 0.011*"nicosia" + 0.010*"walk" + '
  '0.009*"north" + 0.009*"breakfast" + 0.009*"city" + 0.008*"great" + '
  '0.007*"room" + 0.007*"cyprus"'),
 (3,
  '0.014*"gate" + 0.011*"house" + 0.011*"stay" + 0.010*"walking" + '
  '0.010*"nicosia" + 0.008*"city" + 0.008*"home" + 0.008*"selay" + '
  '0.008*"kyrenia" + 0.007*"tourist"'),
 (4,
  '0.010*"city" + 0.005*"northern" + 0.005*"cyprus" + 0.005*"centre" + '
  '0.005*"situated" + 0.005*"ancient" + 0.005*"tourist" + 0.005*"note" + '
  '0.005*"limited" + 0.004*"breakfast"')]


In [21]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.027297 -0.098350       1        1  33.720005
2      0.049189  0.054550       2        1  28.852337
3      0.053142  0.022835       3        1  18.596224
0     -0.058690  0.022460       4        1  15.909916
4     -0.070937 -0.001496       5        1   2.921518, topic_info=          Term       Freq      Total Category  logprob  loglift
46        city  44.000000  44.000000  Default  30.0000  30.0000
996      hotel  36.000000  36.000000  Default  29.0000  29.0000
4         gate  58.000000  58.000000  Default  28.0000  28.0000
33     walking  16.000000  16.000000  Default  27.0000  27.0000
135      walls  15.000000  15.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
79       local   0.369446   9.791197   Topic5  -5.9288   0.2558
132       walk   0.373178  26.059128   Topic5  -5.9187  -0.7130
33     walking   0.370896  16.346088   Topic5  -5.9249  -0.2528
243       town   0.370779  16.745378   Topic5  -5.9252  -0.2772
321  excellent   0.369259  14.181391   Topic5  -5.9293  -0.1151

[363 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
34        1  0.747690           added
769       1  0.852830        adjacent
273       1  0.853526  administration
1068      4  0.728386    affascinante
1069      4  0.728402           amore
...     ...       ...             ...
139       1  0.586995           worth
139       2  0.073374           worth
139       3  0.146749           worth
139       4  0.146749           worth
611       2  0.952038            yard

[498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1, 5])